In [10]:
# Vocab Summary

In [143]:
# Arguments
# GOOGLE API KEYS see https://console.cloud.google.com/
TRANSLATE_API_KEY = "##########################"
TEXTTOSPEECH_API_KEY = "##########################"

# Supported voices
# https://cloud.google.com/text-to-speech/docs/voices?hl=en_US
SPEECH_OUTPUT_LANGUAGE_CODE = "ko-KR"
SPEECH_OUTPUT_LANGUAGE_NAME = "ko-KR-Standard-A"
SPEECH_OUTPUT_LANGUAGE_GENDER = "FEMALE"

# Open Anki in custom location
# open /Applications/Anki.app --args -b ~/anki
ANKI_MEDIA_LOCATION = "/Users/<USER>/anki/<ANKI USER>/collection.media/"


In [144]:
title = "A Sleepyhead Baby Bear"
text = """
한 아기 곰이 숲속에 살았습니다. 
그가 늦잠을 잤고 학교에 늦게 갔습니다. 
그의 반 친구들은 그를 놀렸습니다. 
그가 그의 얼굴을 씻지 않았기 때문이었습니다. 
그가 연못으로 갔고 그의 얼굴을 씻었습니다. 
그의 얼굴이 깨끗해졌습니다. 그가 연못을 보고 웃었습니다. 
연못도 그에게 웃어주었습니다.
"""

In [145]:
# Init Google translate API
import requests

def translate(text):
    resp = requests.get("https://translation.googleapis.com/language/translate/v2?target=en&key=" + TRANSLATE_API_KEY + "&q=" + text)
    return resp.json()["data"]["translations"][0]["translatedText"]

In [146]:
# Anki Import
# 
# See https://foosoft.net/projects/anki-connect/
# Installation
# Open the Install Add-on dialog by selecting Tools | Add-ons | Browse & Install in Anki.
# Input 2055492159 into the text box labeled Code and press the OK button to proceed.
# Restart Anki when prompted to do so in order to complete the installation of AnkiConnect.
#
# For MacOS Users
# Start the Terminal application.
# Execute the following commands in the terminal window:
#  defaults write net.ankiweb.dtop NSAppSleepDisabled -bool true
#  defaults write net.ichi2.anki NSAppSleepDisabled -bool true
#  defaults write org.qt-project.Qt.QtWebEngineCore NSAppSleepDisabled -bool true
# Restart Anki.

import json
import urllib.request

def request(action, **params):
    return {'action': action, 'params': params, 'version': 6}

def invoke(action, **params):
    requestJson = json.dumps(request(action, **params)).encode('utf-8')
    response = json.load(urllib.request.urlopen(urllib.request.Request('http://localhost:8765', requestJson)))
    if len(response) != 2:
        raise Exception('response has an unexpected number of fields')
    if 'error' not in response:
        raise Exception('response is missing required error field')
    if 'result' not in response:
        raise Exception('response is missing required result field')
    if response['error'] is not None:
        raise Exception(response['error'])
    return response['result']

In [147]:
# Get unique words from text
vocabList = set(text.replace("\n", " ").replace(",", " ").replace(".", " ").split(" "));

# Translate
vocabMapwithTranslations = {}
for word in vocabList:
    vocabMapwithTranslations[word] = translate(word)

In [148]:
# Init Google Speach API
import requests
import base64 

url = 'https://texttospeech.googleapis.com/v1beta1/text:synthesize'

def getSpokenText(inputText):
    request_data = {
        "input": {
            "text": inputText
        },
        "voice": {
            "languageCode": SPEECH_OUTPUT_LANGUAGE_CODE,
            "name": SPEECH_OUTPUT_LANGUAGE_NAME,
            "ssmlGender": SPEECH_OUTPUT_LANGUAGE_GENDER
        },
        "audioConfig": {
            "audioEncoding": "MP3"
        }
    }


    resp = requests.post(url, json.dumps(request_data), headers = {"X-Goog-Api-Key": TEXTTOSPEECH_API_KEY})
    decodedBytes = base64.b64decode(resp.json()["audioContent"])
    
    return decodedBytes


In [149]:
import os.path

# open /Applications/Anki.app --args -b ~/anki
# https://docs.ankiweb.net/#/files?id=startup-options

def createMP3File(text):
    filename = ANKI_MEDIA_LOCATION + text + ".mp3"
    if os.path.isfile(filename):
        return text + ".mp3"
    else:
        print("Creating new file: " + text + ".mp3")
        
    mp3data = getSpokenText(text)
    with open(filename, 'w+b') as fp:
        fp.write(mp3data)
    
    return text + ".mp3"
    

In [154]:
invoke('createDeck', deck=title)

for word in vocabMapwithTranslations:
    if not word or not vocabMapwithTranslations[word] or word == vocabMapwithTranslations[word]:
        continue;
    soundFile = createMP3File(word)
    invoke('addNote', note = {
            "deckName": title,
            "modelName": "Basic",
            "fields": {
                "Front": word + "[sound:"+ word + ".mp3]",
                "Back": vocabMapwithTranslations[word]
            }
        })
